<a href="https://colab.research.google.com/github/Followb1ind1y/D2L_Pytorch_Study_Notes/blob/main/06_D2L_Builders_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dive into Deep Learning 中文学习笔记** 
# **6. 深度学习计算 （Builders’ Guide）**

## **6.1. 模型构造（Layers and Modules）**

为了实现一些复杂的网络，我们引入了神经网络 **模块（*Module*）** 的概念。一个模块可以描述一个单一的层，一个由多个层组成的组件，或者整个模型本身。使用模块抽象的一个好处是，它们可以被组合成更大的网络，并且通常是递归的。

从编程的角度来看，一个模块由一个 **类（*Class*）** 来代表。它的任何子类必须定义一个前向传播方法，将其输入转化为输出，并且必须存储任何必要的参数。请注意，有些模块根本不需要任何参数。最后，一个模块必须拥有一个反向传播方法，用于计算梯度。

我们重温一下之前用来实现 MLPs 的代码。下面的代码生成了一个具有 256 个单元的全连接隐藏层和 ReLU 激活的网络，然后是一个具有 10个 单元的全连接输出层（没有激活函数）。

In [66]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])


在这个例子中，我们通过实例化 `nn.Sequential` 来构建我们的模型，并将各层的执行顺序作为参数传递。简而言之，`nn.Sequential` 定义了一种特殊的 `Module`，即在 PyTorch 中展示模块的类。它维护着一个有序的组成模块的列表。请注意，两个完全连接的层中的每一个都是 `Linear` Class 的实例，它本身就是 `Module` 的子类。前向传播（forward）方法也非常简单：它将列表中的每个模块连在一起，将每个模块的输出作为输入传给下一个模块。

### **6.1.1. 自定义 Module（A Custom Module）**

在我们实现自己的自定义模块之前，我们简要地总结一下每个模块必须提供的基本功能：

1. 取得输入数据作为其前向传播方法的参数。

2. 通过让前向传播方法返回一个值来生成一个输出。请注意，输出可能有与输入不同的形状。

3. 计算其输出相对于输入的梯度，这可以通过其反向传播方法获得。

4. 存储并提供对那些执行前向传播计算所需的参数。

5. 根据需要初始化模型参数。

In [67]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

`Module` 类是 `nn` 模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承 `Module` 类构造本节开头提到的多层感知机。这里定义的 MLP 类重载了 `Module` 类的 `__init__` 函数和 `forward` 函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

以上的 MLP 类中无须定义反向传播函数。系统将通过**自动求梯度**而自动生成反向传播所需的 `backward` 函数。

我们可以实例化 MLP 类得到模型变量 `net`。下面的代码初始化 `net` 并传入输入数据 `X` 做一次前向计算。其中，`net(X)` 会调用 MLP 继承自 `Module` 类的 `__call__` 函数，这个函数将调用 MLP 类定义的 `forward` 函数来完成前向计算。

In [68]:
net = MLP()
print(net(X).shape)
print(net(X))

torch.Size([2, 10])
tensor([[ 0.2507,  0.0092,  0.0856,  0.1758,  0.3869, -0.1573,  0.1859,  0.1325,
          0.0644, -0.0105],
        [ 0.1898, -0.0754,  0.0787, -0.0019,  0.3339, -0.0243,  0.2162,  0.2272,
          0.0414, -0.0550]], grad_fn=<AddmmBackward0>)


注意，这里并没有将 `Module` 类命名为 `Layer`（层）或者 `Model`（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如 PyTorch 提供的 `Linear` 类），又可以是一个模型（如这里定义的 MLP 类），或者是模型的一个部分。

### **6.1.2. Sequential 类（The Sequential Module）**

当模型的前向计算为简单串联各个层的计算时，`Sequential` 类可以通过**更加简单**的方式定义模型。这正是 `Sequential` 类的目的：它可以接收一个子模块的有序字典（`OrderedDict`）或者一系列子模块作为参数来逐一添加 `Module` 的实例，而模型的前向计算就是将这些实例**按添加的顺序逐一计算**。

为了建立我们自己的简化 `MySequential`，我们只需要定义两个关键方法：1. 一个将模块逐一追加到列表中的方法。2. 一个前向传播的方法，将一个输入通过模块链，以它们被附加的相同顺序传递。下面的 `MySequential` 类提供了与默认 `Sequential` 类相同的功能。

In [69]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

在 `__init__` 方法中，我们通过调用 `add_modules` 方法添加每个模块。这些模块可以在以后被 children 方法访问。通过这种方式，系统知道所添加的模块，它将正确地初始化每个模块的参数。

当我们的 `MySequential` 的前向传播方法被调用时，每个添加的模块都会按照它们被添加的顺序执行。现在我们可以使用我们的 `MySequential` 类重新实现一个 MLP。

In [70]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

### **6.1.3. 在前向传播法中执行代码（Executing Code in the Forward Propagation Method）**

虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义 `forward` 函数，但直接继承 `Module` 类可以极大地拓展模型构造的灵活性。然而，有时我们可能想加入一些既不是前几层的结果也不是可更新参数的部分。我们称这些为 **恒定参数 (*constant parameters*)**。例如，我们想要一个计算函数 $f(x,w)=c \cdot w^{T}x$ 的层，其中 $x$ 输入，$w$ 是我们的参数，并且 $c$ 是一些指定的常数，在优化过程中不被更新。所以我们实现一个 `FixedHiddenMLP` 类，如下所示:

In [71]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个 `FixedHiddenMLP` 模型中，我们实现了一个隐藏层，其权重（`self.rand_weight`）在实例化时被随机初始化，此后保持不变。这个权重不是一个模型参数，因此它永远不会被反向传播所更新。然后，网络将这个 "固定 "层的输出通过一个全连接层。

In [72]:
net = FixedHiddenMLP()
net(X)

tensor(0.2917, grad_fn=<SumBackward0>)

我们可以混合和匹配各种方式将模块组装在一起:

In [73]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.1069, grad_fn=<SumBackward0>)

## **6.2. 参数管理（Parameter Management）**

一旦我们选择了一个架构并设置了超参数，我们就进入训练循环，我们的目标是找到使损失函数最小的参数值。训练结束后，我们将需要这些参数，以便进行未来的预测。此外，我们有时希望提取这些参数，以便在其他情况下重新使用它们，将我们的模型保存在磁盘上，以便在其他软件中执行，或者用于检查。我们首先来具有一个隐藏层的 MLP 例子。

In [74]:
import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

### **6.2.1. 参数访问（Parameter Access）**

当一个模型通过 `Sequential` 类定义时，我们可以首先通过对模型的索引来访问任何一层，就好像它是一个列表。每个层的参数都很方便地位于其属性中。我们可以按以下方式检查第二个全连接层的参数：

In [75]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.3048,  0.2375, -0.0359,  0.0241,  0.3432, -0.2413,  0.3467, -0.1585]])),
             ('bias', tensor([-0.3508]))])

**6.2.1.1. 目标参数（Targeted Parameters）**

请注意，每个参数都被表示为 `parameter` 类的一个实例。要使用参数，我们首先需要访问底层的数值。有几种方法可以做到这一点。下面的代码从第二个神经网络层中提取偏差，返回一个参数类实例，并进一步访问该参数的值。

In [76]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.3508]))

`Parameter` 是复杂的对象，包含值、梯度和附加信息。这就是为什么我们需要明确地请求该值。除了值之外，每个参数还允许我们访问梯度。因为我们还没有为这个网络调用反向传播，它处于初始状态。

In [77]:
net[2].weight.grad == None

True

**6.2.1.2. 访问所有参数（All Parameters at Once）**

对于 `Sequential` 实例中含模型参数的层，我们可以通过 `Module` 类的 `parameters()` 或者 `named_parameters` 方法来访问所有参数（以迭代器的形式返回），后者除了返回参数 Tensor 外还会返回其名字。

In [78]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

### **6.2.2. 捆绑参数（Tied Parameters）**

通常情况下，我们想在多个层之间共享参数。下面，我们分配了一个全连接层，然后专门使用它的参数来设置另一个层的参数。这里我们需要在访问参数之前运行前向传播 `net(X)`。

In [79]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## **6.3. 参数初始化（Parameter Initialization）**

默认情况下，PyTorch 会从一个根据输入和输出维度计算出来的范围中抽取，统一初始化权重和偏置矩阵。PyTorch 的 `nn.init` 模块提供了多种预设的初始化方法。

In [80]:
import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

### **6.3.1. 内置初始化（Built-in Initialization）**

下面的代码将所有权重参数初始化为标准偏差为0.01的高斯随机变量，而偏置参数则清除为零。

In [81]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0064,  0.0168, -0.0200, -0.0079]), tensor(0.))

我们也可以将所有的参数初始化为一个给定的常量值（比如，1）。

In [82]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以为某些区块应用不同的初始化器。例如，下面我们用 Xavier 初始化器初始化第一层，并将第二层初始化为42的常量值。

In [83]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)
def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5611, -0.0304,  0.2399,  0.1844])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### **6.3.2. 自定义初始化（Custom Initialization）**

有时，我们需要的初始化方法并不是由深度学习框架提供的。在下面的例子中，我们使用以下奇怪的分布为任何权重参数 $w$ 定义一个初始化器。

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [84]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.2129,  0.0000,  9.7765, -5.3081],
        [ 6.7363,  9.6402, -0.0000,  6.7002]], grad_fn=<SliceBackward0>)

请注意，我们总是可以选择直接设置参数:

In [85]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, 10.7765, -4.3081])

## **6.5. 自定义层（Custom Layers）**

深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然 PyTorch 提供了大量常用的层，但有时候我们依然希望自定义层。

### **6.5.1. 不含模型参数的自定义层（Layers without Parameters）**

下面的 `CenteredLayer` 类通过继承 `Module` 类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了 `forward` 函数里。这个层里不含模型参数。

In [86]:
import torch
from torch import nn
from torch.nn import functional as F


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

我们可以实例化这个层，然后做前向计算。

In [87]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

我们也可以用它来构造更复杂的模型：

In [88]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

In [89]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

### **6.5.2. 含模型参数的自定义层（Layers with Parameters）**

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。我们可以使用内置函数来创建参数，它们提供一些基本的管理功能。现在让我们来实现我们自己版本的全连接层。这个层需要两个参数，一个表示权重，另一个表示偏置。在这个实现中，我们把 ReLU 激活作为一个默认值。该层需要两个输入参数：`in_units` 和 `units`，它们分别表示输入和输出的数量。

In [90]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [91]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.8916,  1.1602, -1.0500],
        [-1.2361,  0.8882, -0.2208],
        [ 0.2404,  0.4173, -1.4291],
        [ 0.0339,  1.4962, -0.5210],
        [ 0.3537, -0.1940, -0.8185]], requires_grad=True)

我们可以直接使用自定义层进行前向传播计算。

In [92]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.0567, 0.0000],
        [0.0000, 1.8363, 0.0000]])

我们还可以使用自定义层来构建模型。一旦我们有了这个，我们就可以像内置的完全连接层一样使用它。

In [93]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[17.3037],
        [10.7841]])

## **6.6. 读取和存储（File I/O）**

在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

### **6.6.1. 读写 `Tensor`（Loading and Saving Tensors）**

我们可以直接使用 `save` 函数和 `load` 函数分别存储和读取 Tensor。`save` 使用 Python 的 pickle 实用程序将对象进行序列化，然后将序列化的对象保存到 disk，使用 `save` 可以保存各种对象,包括模型、张量和字典等。而 `load` 使用 pickle unpickle 工具将 pickle 的对象文件反序列化为内存。

In [94]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

现在我们可以把存储文件中的数据读回内存中。

In [95]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以存储一个 `Tensor` 列表并将其读回内存。

In [96]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以写和读一个字典，从字符串映射到 `Tensor`。当我们想读取或写入一个模型中的所有权重时，这很方便。

In [97]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### **6.6.2. 读写模型（Loading and Saving Model Parameters）**

保存单个权重向量（或其他张量）是很有用的，但如果我们想保存（以后再加载）整个模型，就会变得非常乏力了。毕竟，我们可能有数以百计的参数组洒在整个模型中。出于这个原因，深度学习框架提供了内置功能来加载和保存整个网络。需要注意的一个重要细节是，这**保存的是模型参数而不是整个模型**。因此，为了恢复一个模型，我们需要在代码中生成架构，然后从磁盘加载参数。

In [98]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

我们将模型的参数存储为一个名为 "mlp.params "的文件。

In [99]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们实例化了一个原始 MLP 模型的克隆版本。我们没有随机地初始化模型参数，而是直接读取存储在文件中的参数。

In [100]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

由于两个实例有相同的模型参数，相同的输入 $X$ 的计算结果应该是相同的。让我们来验证这一点。

In [101]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## **6.7. GPU计算（GPUs）**

到目前为止，我们一直在使用CPU计算。对复杂的神经网络和大规模的数据来说，使用 CPU 来计算可能不够高效。在本节中，我们将介绍如何使用 NVIDIA GPU来计算。

In [102]:
!nvidia-smi

Thu Feb  9 19:54:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |    794MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **6.7.1. 计算设备（Computing Devices）**

我们可以指定设备，如 CPU 和 GPU，用于存储和计算。默认情况下，`Tensor` 是在主内存中创建的，然后使用 CPU 来计算。

用 `torch.cuda.is_available()` 查看 GPU 是否可用:

In [103]:
torch.cuda.is_available()

True

在 PyTorch 中，CPU 和 GPU 可以通过 `torch.device('cpu')` 和 `torch.device('cuda')` 表示。需要注意的是，`cpu` 设备意味着所有的物理 CPU 和内存。这意味着 PyTorch 的计算将尝试使用所有的 CPU 核心。然而，一个 `gpu` 设备只代表一个卡和相应的内存。如果有多个 GPU，我们使用 `torch.device(f'cuda:{i}')` 来代表第 $i$ 个 GPU（从0开始）。

In [104]:
import torch
from torch import nn

def cpu():
    return torch.device('cpu')
def gpu(i=0):
    return torch.device(f'cuda:{i}')
cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [105]:
def num_gpus():
    return torch.cuda.device_count()
num_gpus()

1

### **6.7.2. `Tensor` 的 GPU 计算（Tensors and GPUs）**

默认情况下，`Tensor` 是在 CPU 上创建的。我们可以查询 `Tensor` 所在的设备：

In [106]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

值得注意的是，只要我们想对多个项进行操作，它们就必须在同一个设备上。例如，如果我们对两个 `Tensor` 求和，我们需要**确保两个参数都在同一个设备上**--否则，框架将不知道在哪里存储结果，甚至不知道如何决定在哪里进行计算。

使用 `.cuda()` 可以将 CPU 上的 `Tensor` 转换（复制）到 GPU 上。

In [107]:
x = x.cuda(0)
x

tensor([1, 2, 3], device='cuda:0')

我们可以通过 `Tensor` 的 `device` 属性来查看该 `Tensor` 所在的设备。

In [108]:
x.device

device(type='cuda', index=0)

我们可以直接在创建的时候就指定设备：

In [109]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([1, 2, 3], device=device)
# or
x = torch.tensor([1, 2, 3]).to(device)
x

tensor([1, 2, 3], device='cuda:0')

### **6.7.3. 神经网络的GPU计算（Neural Networks and GPUs）**

同 `Tensor` 类似，PyTorch 模型也可以通过 `.cuda` 转换到 GPU 上。我们可以通过检查模型的参数的 `device` 属性来查看存放模型的设备。

In [110]:
net = nn.Sequential(nn.LazyLinear(1))
net.cuda()

Sequential(
  (0): LazyLinear(in_features=0, out_features=1, bias=True)
)

In [111]:
X = torch.ones(2, 3, device=device)
net(X)

tensor([[0.9156],
        [0.9156]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [112]:
net[0].weight.data.device

device(type='cuda', index=0)

In [113]:
%%shell
jupyter nbconvert --to html 06_D2L_Builders_Guide.ipynb

[NbConvertApp] Converting notebook 06_D2L_Builders_Guide.ipynb to html
[NbConvertApp] Writing 371583 bytes to 06_D2L_Builders_Guide.html
